<a href="https://colab.research.google.com/github/aztecman/Magic_the_gathering_python_ML/blob/master/MTG_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install mtgjson

    100% |████████████████████████████████| 552kB 6.4MB/s 
  Running setup.py bdist_wheel for mtgjson ... - \ done
  Stored in directory: /root/.cache/pip/wheels/03/6a/52/05f8debcbf116167d65ea247abdd04ae38a49be729cd8f1456
Successfully built mtgjson


In [0]:
# https://pythonhosted.org/mtgjson/
from mtgjson import CardDb

In [0]:
# https://mtgjson.com/
ALL_SETS_X_ZIP_URL = 'http://mtgjson.com/json/AllSets-x.json.zip'
db = CardDb.from_url(ALL_SETS_X_ZIP_URL)

In [0]:
for m_set in db.sets:
  print(m_set, db.sets[m_set].name)

LEA Limited Edition Alpha
LEB Limited Edition Beta
CEI International Collector's Edition
CED Collector's Edition
2ED Unlimited Edition
ARN Arabian Nights
pDRC Dragon Con
ATQ Antiquities
3ED Revised Edition
LEG Legends
DRK The Dark
FEM Fallen Empires
pMEI Media Inserts
pLGM Legend Membership
4ED Fourth Edition
ICE Ice Age
CHR Chronicles
HML Homelands
RQS Rivals Quick Start Set
ALL Alliances
pARL Arena League
pCEL Celebration
MIR Mirage
MGB Multiverse Gift Box
ITP Introductory Two-Player Set
VIS Visions
5ED Fifth Edition
pPOD Portal Demo Game
POR Portal
VAN Vanguard
WTH Weatherlight
pPRE Prerelease Events
TMP Tempest
STH Stronghold
pJGP Judge Gift Program
EXO Exodus
PO2 Portal Second Age
UGL Unglued
pALP Asia Pacific Land Program
USG Urza's Saga
ATH Anthologies
ULG Urza's Legacy
6ED Classic Sixth Edition
PTK Portal Three Kingdoms
UDS Urza's Destiny
pGRU Guru
S99 Starter 1999
pWOR Worlds
pWOS Wizards of the Coast Online Store
MMQ Mercadian Masques
BRB Battle Royale Box Set
pSUS Super Seri

In [0]:
sets_to_skip = ["DD" + chr(i) for i in range(65, 86)] + ['EVG', 'DD2', 'DPA', 'DD3_JVC', 'DD3_GVL', 'DD3_EVG', 'DD3_DVD']
limited_sets = [i for i in db.sets if i not in sets_to_skip]
print(limited_sets)
#limited_sets = [str(i) + 'ED' for i in range(2,10)] + ['M' + str(i) for i in range(10,16)] + ['M19'] 

['LEA', 'LEB', 'CEI', 'CED', '2ED', 'ARN', 'pDRC', 'ATQ', '3ED', 'LEG', 'DRK', 'FEM', 'pMEI', 'pLGM', '4ED', 'ICE', 'CHR', 'HML', 'RQS', 'ALL', 'pARL', 'pCEL', 'MIR', 'MGB', 'ITP', 'VIS', '5ED', 'pPOD', 'POR', 'VAN', 'WTH', 'pPRE', 'TMP', 'STH', 'pJGP', 'EXO', 'PO2', 'UGL', 'pALP', 'USG', 'ATH', 'ULG', '6ED', 'PTK', 'UDS', 'pGRU', 'S99', 'pWOR', 'pWOS', 'MMQ', 'BRB', 'pSUS', 'pFNM', 'pELP', 'NMS', 'S00', 'PCY', 'BTD', 'INV', 'PLS', '7ED', 'pMPR', 'APC', 'ODY', 'DKM', 'TOR', 'JUD', 'ONS', 'LGN', 'SCG', 'pREL', '8ED', 'MRD', 'DST', '5DN', 'CHK', 'UNH', 'BOK', 'SOK', '9ED', 'RAV', 'p2HG', 'pGTW', 'GPT', 'pCMP', 'DIS', 'CST', 'CSP', 'TSP', 'TSB', 'pHHO', 'PLC', 'pPRO', 'pGPX', 'FUT', '10E', 'pMGD', 'pSUM', 'MED', 'LRW', 'pLPA', 'MOR', 'p15A', 'SHM', 'EVE', 'DRB', 'ME2', 'pWPN', 'ALA', 'CON', 'ARB', 'M10', 'V09', 'HOP', 'ME3', 'ZEN', 'H09', 'WWK', 'ROE', 'ARC', 'M11', 'V10', 'SOM', 'PD2', 'ME4', 'MBS', 'NPH', 'CMD', 'M12', 'V11', 'ISD', 'PD3', 'DKA', 'AVR', 'PC2', 'M13', 'V12', 'RTR', 'CM1'

In [0]:
text = ""
for i in limited_sets[:int(len(limited_sets)/2)]:
  for card in db.sets[i].cards:
    try:
      text += card.name + '\n' + card.text + '\n\n'
    except:
      pass
print(len(text))

2247165


In [0]:
import keras
import numpy as np

Using TensorFlow backend.


In [0]:
# Length of extracted character sequences
maxlen = 60

# We sample a new sequence every `step` characters
step = 3

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 749035
Unique characters: 95
Vectorization...


In [0]:
from keras import layers
from keras.layers import Dropout

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
#model.add(Dropout(0.2))
model.add(layers.Dense(len(chars), activation='softmax'))

In [0]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [0]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [0]:
import random
import sys

for epoch in range(1, 60):
    print('epoch', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=50,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.4, 0.6, 0.8]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


epoch 1
Epoch 1/1
749035/749035 [==============================] - 2374s 3ms/step - loss: 1.1787
--- Generating with seed: "or greater.
Orgg can't block creatures with power 3 or great"
------ temperature: 0.2
or greater.
Orgg can't block creatures with power 3 or greater of the top of your control, you may cast that cards and toughness its controller.

Stread Stride
{1}{W}: The next time a creature to its owne that creatures.

Thing Spire
{1}{B}{B}{B}{B}{B}: The next time a creature with that creature to cast.

Thish Spire
Whenever a permanent of the top of your control, you may cast this spell and toughness that cards from your graveyard from your graveyard fr
------ temperature: 0.4
ghness that cards from your graveyard from your graveyard from your graveyard from your graveyard from your graveyard to your mana pool.

Stricker Warpel
As a creature to its owne control.

Tire Sener
{1}{W}: This card and that creature with converted control this strips counter from and puts that that c

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


 except by creatures with first strike. Then shuffle your library.

------ temperature: 0.6
 by creatures with first strike. Then shuffle your library.

Inging Deat
Strack Spire
Whenever a green enden for each creature and only any time you could preve as long as you control that creatures with the top of your control.

Bolless
At the beging of your upkeep, it gets +1/+1 until end of turn.

Trave
Target player gaye.

Wing Spire
Changon can't be blocked as ane to its owne proen to your graveyard to your hand.

Theinge Warn
Whenever a player discar
------ temperature: 0.8
aveyard to your hand.

Theinge Warn
Whenever a player discards two the top of the next end step.

Strive Screngpene
First strike
{1}{W}: Pid of Greventroped enters the battlefield, it can't be blocked except by creatures.

Wall Tue
Enchanted creature geto nonis than mananing dealt damage, counter from Rem Randsing artifact this card onto the battlefield spell. Counter target creature can't or ling. Then that gains first

In [0]:
#TODO:

#encode the data as described here:
#https://github.com/billzorn/mtgencode#tutorial

#plot loss
#double data size
#increase size of NN by one layer
#increase batch size to 100